In [2]:
from pyclausie import ClausIE
cl = ClausIE.get_instance()



In [3]:
import spacy
nlp=spacy.load('en')
doc = nlp("Crile graduated from Ohio Northern University in 1885, and in 1887 received his M.D. from Wooster Medical College which merged to form modern day Case Western Reserve University School of Medicine.He did further study at Vienna, London and Paris. He taught at Wooster from 1889 to 1900. He was professor of clinical medicine at Western Reserve University from 1900 to 1911, and was then made professor of surgery. ")

#sents = ['Barack Hussein Obama II (US /bəˈrɑːk huːˈseɪn oʊˈbɑːmə/; born August 4, 1961) is an American politician who is the 44th and current President of the United States. He is the first African American to hold the office and the first president born outside the continental United States. Born in Honolulu, Hawaii, Obama is a graduate of Columbia University and Harvard Law School, where he was president of the Harvard Law Review. He was a community organizer in Chicago before earning his law degree. He worked as a civil rights attorney and taught constitutional law at the University of Chicago Law School between 1992 and 2004. While serving three terms representing the 13th District in the Illinois Senate from 1997 to 2004, he ran unsuccessfully in the Democratic primary for the United States Hou']

sents=[]

for sent in doc.sents:
    sents.append(sent.text)
    
#print(sents)
    
#print(sents)
triples = cl.extract_triples(sents)
#print(triples)
for triple in triples:
    print("===============")
    print(str(triple).split(",")[1].split("=")[1],str(triple).split(",")[2].split("=")[1], str(triple).split(",")[3].split("=")[1])
    #print(triple)


'Crile' 'graduated' 'from Ohio Northern University in 1885')
'Crile' 'received' 'his M.D. in 1887')
'Crile' 'received' 'his M.D. from Wooster Medical College')
'Crile' 'received' 'his M.D.')
'his' 'has' 'M.D.')
'Wooster Medical College' 'merged' 'to form modern day Case Western Reserve University School of Medicine')
'Wooster Medical College' 'merged' 'to form Case Western Reserve University School of Medicine')
'He' 'did' 'further study at Vienna London and Paris')
'He' 'did' 'further study')
'He' 'taught' 'at Wooster from 1889 to 1900')
'He' 'taught' 'at Wooster')
'He' 'was' 'professor of clinical medicine')
'He' 'was' 'professor at Western Reserve University from 1900 to 1911')
'He' 'was' 'professor')
'He' 'was made' 'professor of surgery')


In [4]:
import json

resource_dict={}
with open('/home/ishani/Documents/GSOC2020/Obama_Json.json') as f:
    data = json.load(f)
    for elem in (data['Resources']):
        resource_dict[elem['@surfaceForm'].lower()]= elem['@URI'].lower()

In [16]:
import spacy
nlp=spacy.load('en')

from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, FOAF


doc = nlp("Barack Hussein Obama II is an American politician who is the 44th and current President of the United States. He is the first African American to hold the office and the first president born outside the continental United States. Born in Honolulu, Hawaii, Obama is a graduate of Columbia University and Harvard Law School, where he was president of the Harvard Law Review. He was a community organizer in Chicago before earning his law degree. He worked as a civil rights attorney and taught constitutional law at the University of Chicago Law School between 1992 and 2004. While serving three terms representing the 13th District in the Illinois Senate from 1997 to 2004, he ran unsuccessfully in the Democratic primary for the United States Hou")

sents=[]

for sent in doc.sents:
    sents.append(sent.text)
    
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

triples = cl.extract_triples(sents)
lemmatizer = WordNetLemmatizer()

"""
Using a handful of white-list of lexicalized verbs
"""


white_dict={}
white_dict['graduate of']='http://dbpedia.org/ontology/college'
white_dict['study at']='http://dbpedia.org/ontology/college'
white_dict['work as']='http://dbpedia.org/ontology/occupation'
white_dict['be a']='rdf:type'
white_dict['be an']='rdf:type'
white_dict['be the']='rdf:type'
nlp=spacy.load('en')

rdfGraph = Graph()

print(white_dict.keys())
#white_dict='work as':'http://dbpedia.org/ontology/occupation', 'be a':'rdf:type', 'be an':'rdf:type', 'be the':'rdf:type'}
for triple in triples:
    #print("===============")
    #print(str(triple).split(",")[1].split("=")[1],str(triple).split(",")[2].split("=")[1], str(triple).split(",")[3].split("=")[1])
    sentence = str(triple).split(",")[1].split("=")[1][1:-1]+ " "+str(str(triple).split(",")[2].split("=")[1])[1:-1]+" "+str(str(triple).split(",")[3].split("=")[1])[1:-2]
    
    s=str(triple).split(",")[1].split("=")[1][1:-1].lower()
    if(s=='he'):
        s='obama'
    p=str(str(triple).split(",")[2].split("=")[1])[1:-1].lower()
    o=str(str(triple).split(",")[3].split("=")[1])[1:-2].lower()
    
    nps=[]
    doc=nlp(s)
    for np in doc.noun_chunks:
        nps.append(np.text)
        
    """
    subject checking 
    """
    
    subjects=[]
    if(len(nps)>0):
        if(nps[0] in resource_dict.keys()):
            subjects.append(resource_dict[nps[0]])
        else:
            for key, elem in resource_dict.items():
                if(key.lower() in nps[0].lower()):
                    subjects.append(resource_dict[key])
        
    """
    predicate checking 
    """
    
    preds=[]
    converted_sentence=[]
    for word in (pos_tag(word_tokenize(sentence))):
        if(str(word[1]).startswith('VB')):
            converted_sentence.append(lemmatizer.lemmatize(str(word[0]), 'v'))
        else:
            converted_sentence.append(word[0])
          
    
    converted_sentence=str(" ".join(converted_sentence))
    
    for verb in list(white_dict.keys()):
        if(verb in converted_sentence):
            preds.append(white_dict[verb])
    

    
    for elem in preds:
        if(elem not in list(white_dict.values())):
            print(elem)
        
    if(len(preds)<1):
        for word in (pos_tag(word_tokenize(sentence))):
            if(str(word[1]).startswith('VB')):
                preds.append(word[0])
           
            
    """
    object checking 
    """
    
    objects=[]
            
    doc=nlp(o)
    nps=[]
    for np in doc.noun_chunks:
        nps.append(np.text)
        
    if(len(nps)>0):
        #print(nps[0])
        for key, elem in resource_dict.items():
            if(key in nps[0]):
                #print(nps[0], resource_dict[key])
                objects.append(resource_dict[key])
            elif(nps[0] in key):
                objects.append(resource_dict[key])
    
   
    if(len(subjects)>0 and len(preds)>0 and len(objects)>0):
        print(subjects[0], " ".join(preds), objects[0])
        subject = URIRef(subjects[0])
        predicate = BNode()
        rdfGraph.add( (predicate, FOAF.name, Literal(preds[0])) )
        obj = URIRef(objects[0])
        rdfGraph.add((subject, predicate, obj))
    

dict_keys(['graduate of', 'study at', 'work as', 'be a', 'be an', 'be the'])
http://dbpedia.org/resource/barack_obama rdf:type rdf:type http://dbpedia.org/resource/united_states
http://dbpedia.org/resource/united_states rdf:type http://dbpedia.org/resource/president_of_the_united_states
http://dbpedia.org/resource/united_states rdf:type http://dbpedia.org/resource/president_of_the_united_states
http://dbpedia.org/resource/barack_obama rdf:type http://dbpedia.org/resource/united_states
http://dbpedia.org/resource/president_of_the_united_states be born http://dbpedia.org/resource/united_states
http://dbpedia.org/resource/barack_obama was http://dbpedia.org/resource/president_of_the_united_states
http://dbpedia.org/resource/barack_obama was http://dbpedia.org/resource/president_of_the_united_states
http://dbpedia.org/resource/barack_obama was http://dbpedia.org/resource/president_of_the_united_states
http://dbpedia.org/resource/barack_obama rdf:type http://dbpedia.org/resource/community_o

In [15]:
print(rdfGraph.serialize(format='turtle'))

b'@prefix ns1: <http://xmlns.com/foaf/0.1/> .\n\n<http://dbpedia.org/resource/barack_obama> [ ns1:name "[\'taught\']" ] <http://dbpedia.org/resource/constitutional_law> ;\n    [ ns1:name "[\'rdf:type\']" ] <http://dbpedia.org/resource/community_organizing> ;\n    [ ns1:name "[\'rdf:type\']" ] <http://dbpedia.org/resource/community_organizing> ;\n    [ ns1:name "[\'rdf:type\']" ] <http://dbpedia.org/resource/united_states> ;\n    [ ns1:name "[\'taught\']" ] <http://dbpedia.org/resource/constitutional_law> ;\n    [ ns1:name "[\'was\']" ] <http://dbpedia.org/resource/president_of_the_united_states> ;\n    [ ns1:name "[\'rdf:type\']" ] <http://dbpedia.org/resource/community_organizing> ;\n    [ ns1:name "[\'was\']" ] <http://dbpedia.org/resource/president_of_the_united_states> ;\n    [ ns1:name "[\'rdf:type\', \'rdf:type\']" ] <http://dbpedia.org/resource/united_states> ;\n    [ ns1:name "[\'http://dbpedia.org/ontology/occupation\']" ] <http://dbpedia.org/resource/civil_and_political_right